# 本文的目录如下：

* 解释使用连续词向量的原因；
* 词向量模型的原理及训练过程；
* 在Tensorflow中实现模型的简单版本，并给出优化的方法。

TensorFlow实现了两个版本的模型：[简单版](https://www.tensorflow.org/code/tensorflow/examples/tutorials/word2vec/word2vec_basic.py)和[正式版](https://www.tensorflow.org/code/tensorflow/models/embedding/word2vec.py)。如果想看源码，可以直接下载。

----

# 为什么要使用Word Embedding

在信号处理领域，图像和音频的输入往往是表示成高维度、密集的向量形式，在图像和音频的应用系统中，如何对输入信息进行编码(Encoding)显得非常重要和关键，这将直接决定了系统的质量。然而，在自然语言处理领域中，传统的做法是将此表示成离散的符号，例如将[cat]表示成[Id537]，而[dog]表示为[Id143].这样做的缺点在于，没有提供足够的信息来体现词语之间的某种关联，例如尽管cat和dog不是一个词，但是却应该有着某种联系(都是属于动物种类)。由于这种一元表示法(One-hot Representation)使得词向量过于稀疏，所以往往需要大量的语料数据才能训练出一个令人满意的模型。而Word Embedding技术则可以解决上述传统方法带来的问题。

![](2.TensorFlow-word2vec_Files/example.jpg)